# GANs: Generative Adversarial Networks
Imagine the following situation: you have a counterfeitor and a cop.   
*  The counterfeitor makes a fake money sample.  Initially, the counterfeitor is very bad at this!
*  The cop examines the fake money sample, compares it with real money, and she determines that the sample is fake.
*  Based on the information from the cop, the counterfeitor improves his performance at making fake money and tries again.
*  The cop examines the new fakes, and she determines - again - that they are indeed fake.
*  The process continues, until at some point, the cop is no longer able to tell the difference between fake and real currency.   

This is the basic outline of how a GAN works.   The GAN starts out with no knowledge of a sample, but through the use of a **generator** (the counterfeitor) and a **discriminator** (the cop), can end up with incredibly realistic versions of images, music, text, etc.

GANs a represent an example of a **unsupervised learning**, in which the model learns features about a dataset without the data being labeled.

GANS are a relatively new idea in machine learning, but they are quite interesting.  Yann LeCun - one of the major figures in machine learning - has called them “The coolest idea in deep learning in the last 20 years.”   Let see if we can understand how they work.

For this workbook, I will rely heavily on the model described in Mike Bernico's book, [Deep Learning Quick Reference](https://www.amazon.com/Deep-Learning-Quick-Reference-optimizing-ebook/dp/B0791JRGPY).

# A simple GAN Model
A simple cartoon of a GAN is shown here (from [this link](https://medium.freecodecamp.org/an-intuitive-introduction-to-generative-adversarial-networks-gans-7a2264a81394)):
![gan](gan.png)

The basic function of the GAN is the following:
* The **generator** network is fed random noise, and outputs a *fake image*.
* The **discriminator** network is fed a single image, as well as a label indicating that the image is either real or fake.   It should output 1 if the image is real, and 0 if the image is fake.

# Implementation
We will design a GAN which will generate fake - but realistic - looking MNIST digits.  For the most part, we will be using techniques that you have seen before.   However, the interplay of the two networks of generator and discriminator make the successful training of a GAN very tricky.   So we will need to add some new features in order to make the training more stable.



# The Discriminator

Lets start with the discriminator.   We have made models like this before.  In fact, the basic idea here is fairly straightforward: a simple CNN which takes 28x28x1 images (like our MNIST dataset) and then produces a single output: 1 if the image is real, and 0 is the image is fake.  

The discriminator we will use is shown below.   You will notice two new features:
* A **Leaky ReLU** layer.  This replaces the ReLU activations that we have used before.  Leaky ReLU allows the pass of a small gradient signal for negative values. 
* A Batch Normalization layer.  Batch norm works by normalizing the input features of a layer to have zero mean and unit variance. Batch norm helps to deal with problems due to poor parameter initialization.

In [ ]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model
from keras.optimizers import Adam
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np

def build_discriminator(img_shape):
    input = Input(img_shape)
    x =Conv2D(32, kernel_size=3, strides=2, padding="same")(input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64, kernel_size=3, strides=2, padding="same")(x)
    x = ZeroPadding2D(padding=((0, 1), (0, 1)))(x)
    x = (LeakyReLU(alpha=0.2))(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(128, kernel_size=3, strides=2, padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(256, kernel_size=3, strides=1, padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    out = Dense(1, activation='sigmoid')(x)

    model = Model(input, out)
    print("-- Discriminator -- ")
    model.summary()
    return model

# The Generator
The generator takes a random vector - in this case a vector of length 100 - and via a series of Keras layers produces an image - in our case a 28x28x1 image.   It uses the Batch Normalization layer, as well as an **UpSampling** layer.  We have previosly used UpSampling layers when we first introduced CNN autoencoders.   The UpSampling layer repeats the rows and columns of the data by size[0] and size[1] respectively.


In [ ]:

def build_generator(noise_shape=(100,)):
    input = Input(noise_shape)
    x = Dense(128 * 7 * 7, activation="relu")(input)
    x = Reshape((7, 7, 128))(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(1, kernel_size=3, padding="same")(x)
    out = Activation("tanh")(x)
    model = Model(input, out)
    print("-- Generator -- ")
    model.summary()
    return model

# Get the data
The following is a helper function to get the MNIST data.   We will end up just using the training image data, and not the test data.   **We won't use the MNIST labels at all**.  The key thing we know about the MNIST training images is that they are **real**.   We will label images below, but the only labels we need are whether the images are **real** (which is only the case if they come from MNIST) or **fake** (which is only the case if the images come from our generator model above).   We don't need to know if the real image is a 0,1,..,9,  just that it is real.

In [ ]:

def load_data():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    return X_train



# Helper function for displaying the generator images
As we train our generator, we will want to inspect the images to see how close they appear to emulating real images.   We will call the function a number of times during each epoch that we train the networks.    The code below uses 25 random vectors of length 100, feeds them into the current version of the generator, and makes 25 fake images.   It does not display them to the screen - it saves them to a directory.

In [ ]:
def save_imgs(generator, epoch, batch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d_%d.png" % (epoch, batch))
    plt.close()


# Build and compile the discriminator and generator

In [ ]:
discriminator = build_discriminator(img_shape=(28, 28, 1))
discriminator.compile(loss='binary_crossentropy',
                               optimizer=Adam(lr=0.0002, beta_1=0.5),
                               metrics=['accuracy'])

generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))


# Training the discriminator
We have everything we need to train the discriminator: we have real images (from MNIST) and fake images (which will come from the generator).  So to train the discriminator, we simply have to feed it batches of labeled (remeber just if they are real or fake) images.   But how do we train the generator?

# Training the generator
The generator needs information to determine how poorly (or well) its fake images are.  To do this, we will use a clever trick:  we will use the output of the same discriminator above!  

To do this, we will need to make a **third** model, which below we call the **combined** model :
*  The combined model uses the generator output, and feeds it into the discriminator.  For this step, we will tell the combined model that these fake images are actually real.
*  During the **generator training process**, we will need to freeze the weights of the discriminator, so that only the weights of the generator are adjusted.   

In [ ]:
z = Input(shape=(100,))
img = generator(z)
discriminator.trainable = False
real = discriminator(img)
combined = Model(z, real)
combined.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

# The Training loop
The training loop below contains the logic of how we fit all 3 models (really just the two: the generator and the discriminator).  Some things to pay attention to in the code below:

1.  There is an outer loop over epochs, and an inner loop over batches.   With a batch size of 32 and the 60k MNIST images, this means we will run the inner loop 1875 times per epoch.
2.  During each batch iteration, we feed the disrimnator 16 real images and 16 fake images.  The labels for these are 1 and 0 respectively.  Notice that we randomly sample (with replacement) the real images.   So we are not using the full data each epoch!   Might be a good idea to revisit this if we were doing it for real....
3.  In the first half of this loop, we run **train_on_batch** to train the **discriminator only**.   We use train_on_batch rather than **fit** because we need to control how we feed data to the model when fitting - train_on_batch allows us to do that.
4.  **After** this, we then send 32 new **fake** images to the **combined** model.   These images are labeled as **real** (even though they are fake).   Remember that the **same** discriminator model is used as in step 3, but for this part of the loop the discriminator weights are frozen to the values that they had at step 3.  **Only the generator** weights are changed during this step.   Since we are asking the disciminator output to be 1, we are adjusting the generator weights so that the generator gets better at producing images which the discriminator will believe are real.
5.  Every 50 batches within each epoch, we generate 50 fake images, using the current version of the generator model.
6.  As the training progresses through epochs/batches, examine how the discriminatr loss and accuracy, as well as the generator loss change.  Note: train_on_batch for the discriminator returns both the loss as well as the metric (which is the accuracy in this case).


**NOTE**: Each epoch will take **alot** of time!   So you probably will want to stop it after two epoch or so.   If you want to see some cool images after 1,2,3,4 epochs, submit the script  **pbs_gan_gpu.sh** to the pbs batch system.   In about 30 minutes you will have some amazingly real digit images in the images/ directory!

In [ ]:
X_train = load_data()

epochs=2        # you need about 40 epochs to get good images
batch_size=32
save_interval=1

num_examples = X_train.shape[0]
num_batches = int(num_examples / float(batch_size))
print('Number of examples: ', num_examples)
print('Number of Batches: ', num_batches)
print('Number of epochs: ', epochs)

half_batch = int(batch_size / 2)

for epoch in range(epochs + 1):
    for batch in range(num_batches):

            # noise images for the batch
        noise = np.random.normal(0, 1, (half_batch, 100))
        fake_images = generator.predict(noise)
        fake_labels = np.zeros((half_batch, 1))

            # real images for batch
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_images = X_train[idx]
        real_labels = np.ones((half_batch, 1))

            # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, 100))
            # Train the generator
        g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))

            # Plot the progress
        print("Epoch %d Batch %d/%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                  (epoch,batch, num_batches, d_loss[0], 100 * d_loss[1], g_loss))

        if batch % 50 == 0:
            save_imgs(generator, epoch, batch)


# A Cool Use Case for GANs



From: https://towardsdatascience.com/generative-adversarial-networks-gans-a-beginners-guide-5b38eceece24

"In addition to generating beautiful pictures, an approach for semi-supervised learning with GANs has been developed that involves the discriminator producing an additional output indicating the label of the input. This approach enables cutting edge results on datasets with very few labeled examples. On MNIST, for example, 99.1% accuracy has been achieved with only 10 labeled examples per class with a fully connected neural network — a result that’s very close to the best known results with fully supervised approaches using all 60,000 labeled examples. This is extremely promising because labeled examples can be quite expensive to obtain in practice."

Here is an example of how this idea is implemented in practice:
https://towardsdatascience.com/semi-supervised-learning-with-gans-9f3cb128c5e

# Extra Credit - 5 pts!
This could be a bit difficult:
* Run the python version of the above code.  You can find a .py file and a .sh file in this directory to start with.  The shell script can be used with **qsub** to submit the script to a GPU.
* Modify the python code to save a version of the generator model.  You will need to train the GAN for about 40 epochs to generate realistic images.   This will take about 30 minutes using a GPU.  It would make sense to use something like the file naming procedure used for the images to name your model (meaning you could save the model every epoch, and label the filename using the current epoch number).
* Use a version of the MNIST classsifier that you made in Assignment 10.  You should have a saved model of the trained version of that classifier - copy it here.  If you can't find it, a version that I made can be found in the scratch area:

/fs/scratch/PAS1495/physics6820/GAN/fully_trained_model_cnn_conv_2_2_2_2.h5

* Load both of these models (the CNN MNIST classifier and the generator model).  Then do the follwoing:  
    * Use the GAN generator model to generate 10000 fake digits
    * Feed them into your MNIST classifier and make two plots: 
        * A histogram of which digit your fakes were classified as.  
        * The probability of your chosen digits. (You could also see if this varies by digit but that could be overkill!)